# Hong Kong Weather data Preparation & Data Quality check

Here I aggregate 38 datasets coming from different observatory for daily Temperature, Humidity and Rainfall data. For each dataset, I check the number and the percentage of invalid observations (value of \*\*\*).

After aggregating, a global dataset by the information is created, so that we have **three global datasets** in the end.

In [2]:
# Necessary modules
import pandas as pd  # type: ignore
import time
import datetime
import os
import glob
import numpy as np # type: ignore

## (1) Hong Kong Maximal Temperature Dataset

In [ ]:
# Public Data Downloaded from HK Data Gov website
# CSV File list in 0_Data directory:
list_files = glob.glob("../0_Data/HK/Temperature/*.csv")

# Aggregated Data templete:
Agg_Data = pd.DataFrame()
Data_summary = pd.DataFrame()

for Data_by_dist in list_files:
    
    Max_region=pd.read_csv(Data_by_dist, 
                       sep=",", skiprows=3, header = None)

    # LMMAXT_HKP_: Full Data before any filtering
    LMMAXT_HKP_ = pd.DataFrame(Max_region)
    LMMAXT_HKP_.columns = ['year', 'month', 'day', 'value', 'Flag']
    
    # Data Quality indicators:
    Obs_summary = pd.DataFrame({'Dist': [Data_by_dist.split("Data/Temperature/")[1].split("_")[1]],
                                'NObs_Dist': [LMMAXT_HKP_.shape[0]],
                                'NObs_Flagged': [LMMAXT_HKP_.loc[LMMAXT_HKP_['Flag']=="#"].shape[0]],
                                'NObs_Starred': [LMMAXT_HKP_.loc[LMMAXT_HKP_['value']=="***"].shape[0]],
                                'NObs_Str_Flagged': [LMMAXT_HKP_.loc[(LMMAXT_HKP_['value']=="***")&(LMMAXT_HKP_['Flag']=="#")].shape[0]]                          
                                #'NObs_2007': LMMAXT_HKP_.loc[LMMAXT_HKP_['year']=='2007'].shape[0],
                                #'NObs_St_or_2007': LMMAXT_HKP_.loc[(LMMAXT_HKP_['year']=='2007')|(LMMAXT_HKP_['value']=="***")].shape[0]
                                })
   
    Obs_summary['Starred_Pct(%)'] = round(100*Obs_summary['NObs_Starred']/Obs_summary['NObs_Dist'],2)
    #Obs_summary['Tot_Purge_Pct(%)'] = round(100*Obs_summary['NObs_St_or_2007']/Obs_summary['NObs_Dist'],2)
        
    ## Data Filter ##
    # LMMAXT_HKP_2: Filtered Data
    # F1: Delete the last 3 lines:
    LMMAXT_HKP_2 = LMMAXT_HKP_.iloc[:-3]
    LMMAXT_HKP_2.reset_index(inplace=True, drop=True) 
    LMMAXT_HKP_2.loc[:,'month'] = LMMAXT_HKP_2['month'].values.astype(int)
    LMMAXT_HKP_2.loc[:,'day'] = LMMAXT_HKP_2['day'].values.astype(int)
    LMMAXT_HKP_2 = LMMAXT_HKP_2.loc[LMMAXT_HKP_2['value']!="***"]
    LMMAXT_HKP_2.reset_index(inplace=True, drop=True) 
    LMMAXT_HKP_2.loc[:,'value'] = LMMAXT_HKP_2['value'].values.astype(float)

    # F2: Delete data of 2007 since incomplete (only from october)
    #LMMAXT_HKP_2 = LMMAXT_HKP_2.loc[LMMAXT_HKP_2['year']!='2007']
    
    # Insert the Observatory Name:
    LMMAXT_HKP_2['Dist'] = Data_by_dist.split("Data/Temperature/")[1].split("_")[1]

    # Date parsing 
    LMMAXT_HKP_date = pd.to_datetime(LMMAXT_HKP_2[['year','month', 'day']])
    #LMMAXT_HKP_date = LMMAXT_HKP_date.dt.dayofyear
    
    # Concatenate parsed date + Table 
    # LMMAXT_HKP_3: Concatenated Data
    LMMAXT_HKP_3 = pd.concat([LMMAXT_HKP_date, 
                              LMMAXT_HKP_2[['Dist','year','value', 'Flag']]],
            axis = 1)
    LMMAXT_HKP_3 = LMMAXT_HKP_3.rename(columns = {0: "Date"})
    
    
    # Information on the observed period by Observatory:
    Obs_summary['StartDate'] = LMMAXT_HKP_date.min()
    Obs_summary['EndDate'] = LMMAXT_HKP_date.max()
    
    Data_summary = pd.concat([Data_summary, Obs_summary], axis = 0)
    
    Agg_Data = pd.concat([Agg_Data, LMMAXT_HKP_3], axis = 0, ignore_index=True)

Check if all the lines are completed, computing the percentage of null values:

In [15]:
round(100*Agg_Data.loc[Agg_Data['value'].isna()!=False].shape[0]/Agg_Data.shape[0],1)

0.0

Total number of observations before filtering (From all observatories):

In [16]:
total_obs = Data_summary['NObs_Dist'].sum()
print(total_obs)

362334


Total number of observations after filtering the starred observations (From all observatories):

In [17]:
Agg_Data.shape[0]

358380

Checking the total number of observations, given that we applied 2 filters and 3 lines deletion containing some suppementary informations at the end of each table:

In [20]:
total_obs -Data_summary['NObs_Starred'].sum() -(3*38)

np.int64(358380)

Distribution by observatories:

In [21]:
Data_Composition = pd.DataFrame({
    'Observatory':Data_summary['Dist'].values,
    'NObs':Data_summary['NObs_Dist'].values,
    'Percentage(%)To_TotalOBS':round((Data_summary['NObs_Dist']/total_obs)*100,2),
    'ObservedPeriod(#Year)':[np.ceil(td/np.timedelta64(1, 'D')).astype(int) for td in (Data_summary['EndDate']-Data_summary['StartDate'])/(30*12)] 
})

Data_Composition.sort_values(by=['Percentage(%)To_TotalOBS','ObservedPeriod(#Year)'], 
                             ascending = False,
                             inplace = True)
Data_Composition.reset_index(inplace=True, drop = True)
print(Data_Composition.head())
print(Data_Composition.tail())

  Observatory   NObs  Percentage(%)To_TotalOBS  ObservedPeriod(#Year)
0         HKO  49006                     13.53                    144
1         SHA  14764                      4.07                     41
2         LFS  14399                      3.97                     40
3         TKL  13425                      3.71                     38
4         HKS  12999                      3.59                     37
   Observatory  NObs  Percentage(%)To_TotalOBS  ObservedPeriod(#Year)
33         TY1  5298                      1.46                     15
34          TW  5201                      1.44                     15
35         SE1  3730                      1.03                     11
36         YLP  3637                      1.00                     11
37         CWB  2285                      0.63                      7


In [22]:
print(round(Data_Composition['ObservedPeriod(#Year)'].mean(),1))

27.2


We see that the observatory which logged Temperature data for the longest period in Hong Kong is the one in Hong Kong Observatory in Tsim Sha Tsui area (HKO, for 144 years). It composes of about 14% of the total dataset. The shortest observated period is given by Clear Water Bay Observatory (CWB, for 7 years). The mean value of the observed period over 38 observatories is 27 years.

When we look for the percentage of the invalid lines (containing \*\*\* value), the observatory providing the highest data quality issue is Ngong Ping Observatory (5.5%). The highest invalid data ratio (5.5%) is still relatively low. It goes down to 0% (only one line deleted) when it comes to Hong Kong Observatory which is the most historical observatory. 

In [25]:
print(Data_summary.iloc[:,[0,1,3,5]].sort_values(by='Starred_Pct(%)', ascending=False).head())
print("...")
print(Data_summary.iloc[:,[0,1,3,5]].sort_values(by='Starred_Pct(%)', ascending=False).tail())

  Dist  NObs_Dist  NObs_Starred  Starred_Pct(%)
0  NGP       7855           432            5.50
0  SEK      10320           439            4.25
0  WGL      12968           520            4.01
0  JKB      12147           412            3.39
0  TPO       8710           293            3.36
...
  Dist  NObs_Dist  NObs_Starred  Starred_Pct(%)
0  TY1       5298             2            0.04
0  SSH       7520             3            0.04
0   TW       5201             2            0.04
0  WTS       5816             1            0.02
0  HKO      49006             1            0.00


Without access to the definition of these flags, I have used all numerical values including the ones flagged to '#' in Data Completeness column if it contains a normal numerical value other than '***'.

As of Temperature dataset, the percentage of observations filled with '***' takes less than 1% for 71% of 38 Hong Kong districts. 

In [26]:
round(100*Data_summary.loc[Data_summary['Starred_Pct(%)']<1].shape[0]/Data_summary.shape[0],0)

71.0

Save the aggregated Temperature data in Wrangled directory:

In [27]:
treated_data_rep = r'../0_Data/wrangled/' 
if not os.path.exists(treated_data_rep):
    os.makedirs(treated_data_rep)
Agg_Data.to_pickle(treated_data_rep+"Temperature_AGG.pkl")

## (2) Hong Kong Mean Humidity Dataset

In [ ]:
# Public Data Downloaded from HK Data Gov website
# CSV File list in 0_Data directory:
list_files = glob.glob("../0_Data/HK/Humidity/*.csv")
    
Max_region=pd.read_csv(list_files[0], 
                    sep=",", skiprows=3, header = None)

# LMMAXT_HKP_: Full Data before any filtering
LMMAXT_HKP_ = pd.DataFrame(Max_region)
LMMAXT_HKP_.columns = ['year', 'month', 'day', 'value', 'Flag']

print(LMMAXT_HKP_.head())
print(LMMAXT_HKP_.tail())

   year  month  day  value Flag
0  1947    1.0  1.0   85.0    C
1  1947    1.0  2.0   86.0    C
2  1947    1.0  3.0   84.0    C
3  1947    1.0  4.0   85.0    C
4  1947    1.0  5.0   85.0    C
                          year  month   day  value Flag
28547                     2025    2.0  27.0   76.0    C
28548                     2025    2.0  28.0   77.0    C
28549     *** 沒有數據/unavailable    NaN   NaN    NaN  NaN
28550  # 數據不完整/data incomplete    NaN   NaN    NaN  NaN
28551     C 數據完整/data Complete    NaN   NaN    NaN  NaN


The structure of data seems exactly the same as Max Temperature dataset. Therefore, we proceed the wrangling with the same code. 

In [75]:

# Aggregated Data templete:
Agg_Data = pd.DataFrame()
Data_summary = pd.DataFrame()

for Data_by_dist in list_files:
    
    Max_region=pd.read_csv(Data_by_dist, 
                       sep=",", skiprows=3, header = None)

    # LMMAXT_HKP_: Full Data before any filtering
    LMMAXT_HKP_ = pd.DataFrame(Max_region)
    LMMAXT_HKP_.columns = ['year', 'month', 'day', 'value', 'Flag']
    
    # Data Quality indicators:
    Obs_summary = pd.DataFrame({'Dist': [Data_by_dist.split("Data/Humidity/")[1].split("_")[1]],
                                'NObs_Dist': [LMMAXT_HKP_.shape[0]],
                                'NObs_Flagged': [LMMAXT_HKP_.loc[LMMAXT_HKP_['Flag']=="#"].shape[0]],
                                'NObs_Starred': [LMMAXT_HKP_.loc[LMMAXT_HKP_['value']=="***"].shape[0]],
                                'NObs_Str_Flagged': [LMMAXT_HKP_.loc[(LMMAXT_HKP_['value']=="***")&(LMMAXT_HKP_['Flag']=="#")].shape[0]]                          
                                #'NObs_2007': LMMAXT_HKP_.loc[LMMAXT_HKP_['year']=='2007'].shape[0],
                                #'NObs_St_or_2007': LMMAXT_HKP_.loc[(LMMAXT_HKP_['year']=='2007')|(LMMAXT_HKP_['value']=="***")].shape[0]
                                })
   
    Obs_summary['Starred_Pct(%)'] = round(100*Obs_summary['NObs_Starred']/Obs_summary['NObs_Dist'],2)
    #Obs_summary['Tot_Purge_Pct(%)'] = round(100*Obs_summary['NObs_St_or_2007']/Obs_summary['NObs_Dist'],2)
        
    ## Data Filter ##
    # LMMAXT_HKP_2: Filtered Data
    # F1: Delete the last 3 lines:
    LMMAXT_HKP_2 = LMMAXT_HKP_.iloc[:-3]
    LMMAXT_HKP_2.reset_index(inplace=True, drop=True) 
    LMMAXT_HKP_2.loc[:,'month'] = LMMAXT_HKP_2['month'].values.astype(int)
    LMMAXT_HKP_2.loc[:,'day'] = LMMAXT_HKP_2['day'].values.astype(int)
    LMMAXT_HKP_2 = LMMAXT_HKP_2.loc[LMMAXT_HKP_2['value']!="***"]
    LMMAXT_HKP_2.reset_index(inplace=True, drop=True) 
    LMMAXT_HKP_2.loc[:,'value'] = LMMAXT_HKP_2['value'].values.astype(float)
    
    # Insert the Observatory Name:
    LMMAXT_HKP_2['Dist'] = Data_by_dist.split("Data/Humidity/")[1].split("_")[1]

    # Date parsing 
    LMMAXT_HKP_date = pd.to_datetime(LMMAXT_HKP_2[['year','month', 'day']])
    #LMMAXT_HKP_date = LMMAXT_HKP_date.dt.dayofyear
    
    # Concatenate parsed date + Table 
    # LMMAXT_HKP_3: Concatenated Data
    LMMAXT_HKP_3 = pd.concat([LMMAXT_HKP_date, 
                              LMMAXT_HKP_2[['Dist','year','value', 'Flag']]],
            axis = 1)
    LMMAXT_HKP_3 = LMMAXT_HKP_3.rename(columns = {0: "Date"})
    
    
    # Information on the observed period by Observatory:
    Obs_summary['StartDate'] = LMMAXT_HKP_date.min()
    Obs_summary['EndDate'] = LMMAXT_HKP_date.max()
    
    Data_summary = pd.concat([Data_summary, Obs_summary], axis = 0)
    
    Agg_Data = pd.concat([Agg_Data, LMMAXT_HKP_3], axis = 0, ignore_index=True)

In [95]:
total_obs = Data_summary['NObs_Dist'].sum()
Data_Composition = pd.DataFrame({
    'Observatory':Data_summary['Dist'].values,
    'NObs':Data_summary['NObs_Dist'].values,
    'Percentage(%)To_TotalOBS':round((Data_summary['NObs_Dist']/total_obs)*100,2),
    'ObservedPeriod(#Year)':[np.ceil(td/np.timedelta64(1, 'D')).astype(int) for td in (Data_summary['EndDate']-Data_summary['StartDate'])/(30*12)] 
})

Data_Composition.sort_values(by=['Percentage(%)To_TotalOBS','ObservedPeriod(#Year)'], 
                             ascending = False,
                             inplace = True)
Data_Composition.reset_index(inplace=True, drop = True)

print(f"(1) NA value percentage (%): {round(100*Agg_Data.loc[Agg_Data['value'].isna()!=False].shape[0]/Agg_Data.shape[0],1)} % \n")
print(f"(2-1) Total number of observations (From Summary table sum-up): {total_obs}) \n")
print(f"(2-2) Total number of observations (From Aggregated data number of row counting): {Agg_Data.shape[0]} \n")
print(f"(3) The number of observations after filter application (From Summary table sum-up deducted the # of filtered lines): {total_obs - Data_summary['NObs_Starred'].sum() -(3*24)} \n")
print("(4) Humidity data observed period: \n")
print(Data_Composition.head())
print(Data_Composition.tail())
print(f"\n (5) Humidity data mean observed period: {round(Data_Composition['ObservedPeriod(#Year)'].mean(),1)} year")

(1) NA value percentage (%): 0.0 % 

(2-1) Total number of observations (From Summary table sum-up): 249419) 

(2-2) Total number of observations (From Aggregated data number of row counting): 241237 

(3) The number of observations after filter application (From Summary table sum-up deducted the # of filtered lines): 241237 

(4) Humidity data observed period: 

  Observatory   NObs  Percentage(%)To_TotalOBS  ObservedPeriod(#Year)
0         HKO  28552                     11.45                     80
1         SHA  14764                      5.92                     41
2         LFS  14399                      5.77                     40
3         TKL  13425                      5.38                     38
4         HKS  12999                      5.21                     37
   Observatory  NObs  Percentage(%)To_TotalOBS  ObservedPeriod(#Year)
19         TU1  6637                      2.66                     19
20         KSC  6059                      2.43                     17
21  

For Humidity data, the mean observed period of all the observatories is about 29 years, similar to Temperature data but slightly longer. The maximal observed period is 80 years, also for Hong Kong Observatory. 

In contrary, the percentage of null values (\*\*\*) are higher for Humidity dataset. Its maximal value is observed up to 17% (vs. 5.5% for Temperature dataset). The data quality, at least as of the starred observations, is also very good for the most historical observatory, Hong Kong Observatory. The other observatory which has 0 starred observation is Hong Kong International Airport (HKA).

Data Quality in general is worse for Humidity dataset, when it comes to the percentage of observatories having less than 1% of their dataset filled by starred lines. It only counts 50% of all the observatories, in contrary to Temperature dataset (71%).

In [102]:
print("(6) Missing values by Observatory: \n")
print(Data_summary.iloc[:,[0,1,3,5]].sort_values(by='Starred_Pct(%)', ascending=False).head())
print("\n...\n")
print(Data_summary.iloc[:,[0,1,3,5]].sort_values(by='Starred_Pct(%)', ascending=False).tail())
print(f"\n (7) Percentage of observatories with less than 1% of missing values: {round(100*Data_summary.loc[Data_summary['Starred_Pct(%)']<1].shape[0]/Data_summary.shape[0],0)} %")


(6) Missing values by Observatory: 

  Dist  NObs_Dist  NObs_Starred  Starred_Pct(%)
0  JKB      12147          2101           17.30
0  LFS      14399          1857           12.90
0  WGL      12968           872            6.72
0  SEK      10320           586            5.68
0  TKL      13425           580            4.32

...

  Dist  NObs_Dist  NObs_Starred  Starred_Pct(%)
0   TW       5201             8            0.15
0  YCT       1068             1            0.09
0   KP      11934             9            0.08
0  HKO      28552             0            0.00
0  HKA      10138             0            0.00

 (7) Percentage of observatories with less than 1% of missing values: 50.0 %


Save the aggregated Humidity data in Wrangled directory:

In [103]:
treated_data_rep = r'../0_Data/wrangled/' 
if not os.path.exists(treated_data_rep):
    os.makedirs(treated_data_rep)
Agg_Data.to_pickle(treated_data_rep+"Humidity_AGG.pkl")

# (3) Hong Kong Rainfall Dataset

In [ ]:
# Public Data Downloaded from HK Data Gov website
# CSV File list in 0_Data directory:
list_files = glob.glob("../0_Data/HK/Rainfall/*.csv")
    
Max_region=pd.read_csv(list_files[0], 
                    sep=",", skiprows=3, header = None)

# LMMAXT_HKP_: Full Data before any filtering
LMMAXT_HKP_ = pd.DataFrame(Max_region)
LMMAXT_HKP_.columns = ['year', 'month', 'day', 'value', 'Flag']

print(LMMAXT_HKP_.head())
print(LMMAXT_HKP_.tail())

   year  month  day value Flag
0  2010    4.0  1.0   0.0    C
1  2010    4.0  2.0   4.0    C
2  2010    4.0  3.0   0.0    C
3  2010    4.0  4.0   0.0    #
4  2010    4.0  5.0   ***  NaN
                         year  month   day value Flag
5446                     2025    2.0  27.0   0.0    C
5447                     2025    2.0  28.0   0.0    C
5448     *** 沒有數據/unavailable    NaN   NaN   NaN  NaN
5449  # 數據不完整/data incomplete    NaN   NaN   NaN  NaN
5450     C 數據完整/data Complete    NaN   NaN   NaN  NaN


Also the same structure. I have to delete 2025 datasets since these are already included in the whole period dataset.

In [107]:
list_a_supprimer = [x for x in list_files if "2025" in x]
for files in list_a_supprimer:
    os.remove(files)

For Rainfall data, there were some anormal observations with 'Trace' as value. For example, for Hong Kong Observatory Rainfall dataset, there were 6856 observations filled with 'Trace' (14% of the total dataset):

In [ ]:
Max_region=pd.read_csv("../0_Data/HK/Rainfall/daily_HKO_RF_ALL.csv", 
                    sep=",", skiprows=3, header = None)
LMMAXT_HKP_ = pd.DataFrame(Max_region)
LMMAXT_HKP_.columns = ['year', 'month', 'day', 'value', 'Flag']
print(round(100*LMMAXT_HKP_.loc[LMMAXT_HKP_['value']=="Trace"].shape[0]/LMMAXT_HKP_.shape[0],2))
LMMAXT_HKP_.loc[LMMAXT_HKP_['value']=="Trace"]

14.01


,year,month,day,value,Flag
280,1884,12.0,6.0,Trace,C
281,1884,12.0,7.0,Trace,C
12107,1917,4.0,24.0,Trace,C
12122,1917,5.0,9.0,Trace,C
12123,1917,5.0,10.0,Trace,C
...,...,...,...,...,...
48929,2025,2.0,15.0,Trace,C
48935,2025,2.0,21.0,Trace,C
48936,2025,2.0,22.0,Trace,C
48937,2025,2.0,23.0,Trace,C


In [ ]:
list_files = glob.glob("../0_Data/HK/Rainfall/*.csv")
# Aggregated Data templete:
Agg_Data = pd.DataFrame()
Data_summary = pd.DataFrame()

for Data_by_dist in list_files:
    
    Max_region=pd.read_csv(Data_by_dist, 
                       sep=",", skiprows=3, header = None)

    # LMMAXT_HKP_: Full Data before any filtering
    LMMAXT_HKP_ = pd.DataFrame(Max_region)
    LMMAXT_HKP_.columns = ['year', 'month', 'day', 'value', 'Flag']
    
    # Data Quality indicators:
    Obs_summary = pd.DataFrame({'Dist': [Data_by_dist.split("Data/Rainfall/")[1].split("_")[1]],
                                'NObs_Dist': [LMMAXT_HKP_.shape[0]],
                                'NObs_Flagged': [LMMAXT_HKP_.loc[LMMAXT_HKP_['Flag']=="#"].shape[0]],
                                'NObs_Starred': [LMMAXT_HKP_.loc[LMMAXT_HKP_['value']=="***"].shape[0]],
                                'NObs_Str_Flagged': [LMMAXT_HKP_.loc[(LMMAXT_HKP_['value']=="***")&(LMMAXT_HKP_['Flag']=="#")].shape[0]],                          
                                'NObs_Trace': [LMMAXT_HKP_.loc[LMMAXT_HKP_['value']=='Trace'].shape[0]],
                                'NObs_Purged': [LMMAXT_HKP_.loc[(LMMAXT_HKP_['value'].isin(["***", "Trace"])==True)|((LMMAXT_HKP_['month'].isna()&LMMAXT_HKP_['value'].isna())==True)].shape[0]]
                                })
   
    Obs_summary['Starred_Pct(%)'] = round(100*Obs_summary['NObs_Starred']/Obs_summary['NObs_Dist'],2)
    Obs_summary['Tot_Purge_Pct(%)'] = round(100*Obs_summary['NObs_Purged']/Obs_summary['NObs_Dist'],2)
        
    ## Data Filter ##
    # LMMAXT_HKP_2: Filtered Data
    # F1: Delete the last lines containing comments:
    LMMAXT_HKP_2 = LMMAXT_HKP_.loc[(LMMAXT_HKP_['month'].isna()&LMMAXT_HKP_['value'].isna())==False]
    LMMAXT_HKP_2.reset_index(inplace=True, drop=True) 
    
    # F2: Delete the starred obs or 'Trace' valued lines:
    LMMAXT_HKP_2 = LMMAXT_HKP_2.loc[LMMAXT_HKP_2['value'].isin(["***", "Trace"])==False]
    LMMAXT_HKP_2.reset_index(inplace=True, drop=True) 
    
    LMMAXT_HKP_2.loc[:,'month'] = LMMAXT_HKP_2['month'].values.astype(int)
    LMMAXT_HKP_2.loc[:,'day'] = LMMAXT_HKP_2['day'].values.astype(int)
    LMMAXT_HKP_2.loc[:,'value'] = LMMAXT_HKP_2['value'].values.astype(float)
    
    # Insert the Observatory Name:
    LMMAXT_HKP_2['Dist'] = Data_by_dist.split("Data/Rainfall/")[1].split("_")[1]

    # Date parsing 
    LMMAXT_HKP_date = pd.to_datetime(LMMAXT_HKP_2[['year','month', 'day']])
    #LMMAXT_HKP_date = LMMAXT_HKP_date.dt.dayofyear
    
    # Concatenate parsed date + Table 
    # LMMAXT_HKP_3: Concatenated Data
    LMMAXT_HKP_3 = pd.concat([LMMAXT_HKP_date, 
                              LMMAXT_HKP_2[['Dist','year','value', 'Flag']]],
            axis = 1)
    LMMAXT_HKP_3 = LMMAXT_HKP_3.rename(columns = {0: "Date"})
    
    
    # Information on the observed period by Observatory:
    Obs_summary['StartDate'] = LMMAXT_HKP_date.min()
    Obs_summary['EndDate'] = LMMAXT_HKP_date.max()
    
    Data_summary = pd.concat([Data_summary, Obs_summary], axis = 0)
    
    Agg_Data = pd.concat([Agg_Data, LMMAXT_HKP_3], axis = 0, ignore_index=True)
    

While looking into the Data Filtering summaries, I realize that 'Trace' values are observed only in Hong Kong Observatory dataset:

In [159]:
Data_summary

,Dist,NObs_Dist,NObs_Flagged,NObs_Starred,NObs_Str_Flagged,NObs_Trace,NObs_Purged,Starred_Pct(%),Tot_Purge_Pct(%),StartDate,EndDate
0,SSP,5451,70,44,0,0,47,0.81,0.86,2010-04-01,2025-02-28
0,KSC,6059,190,109,0,0,112,1.80,1.85,2008-08-01,2025-02-28
0,CCH,12025,140,204,0,0,207,1.70,1.72,1992-04-01,2025-02-28
0,TYW,10747,283,229,0,0,232,2.13,2.16,1995-10-01,2025-02-28
0,SEK,10320,92,493,0,0,496,4.78,4.81,1996-12-01,2025-02-28
0,CPH,8220,21,10,0,0,13,0.12,0.16,2002-09-01,2025-02-28
0,VP1,7642,130,141,0,0,144,1.85,1.88,2004-04-01,2025-02-28
0,WGL,12968,378,701,0,0,704,5.41,5.43,1989-09-01,2025-02-28
0,TMS,10258,165,586,0,0,589,5.71,5.74,1997-02-01,2025-02-28
0,JKB,12116,155,443,0,0,446,3.66,3.68,1992-01-01,2025-02-28


In [161]:
total_obs = Data_summary['NObs_Dist'].sum()
Data_Composition = pd.DataFrame({
    'Observatory':Data_summary['Dist'].values,
    'NObs':Data_summary['NObs_Dist'].values,
    'Percentage(%)To_TotalOBS':round((Data_summary['NObs_Dist']/total_obs)*100,2),
    'ObservedPeriod(#Year)':[np.ceil(td/np.timedelta64(1, 'D')).astype(int) for td in (Data_summary['EndDate']-Data_summary['StartDate'])/(30*12)] 
})

Data_Composition.sort_values(by=['Percentage(%)To_TotalOBS','ObservedPeriod(#Year)'], 
                             ascending = False,
                             inplace = True)
Data_Composition.reset_index(inplace=True, drop = True)

print(f"(1) NA value percentage (%): {round(100*Agg_Data.loc[Agg_Data['value'].isna()!=False].shape[0]/Agg_Data.shape[0],1)} % \n")
print(f"(2-1) Total number of observations (From Summary table sum-up): {total_obs}) \n")
print(f"(2-2) Total number of observations (From Aggregated data number of row counting): {Agg_Data.shape[0]} \n")
print(f"(3) The number of observations after filter application (From Summary table sum-up deducted the # of filtered lines): {total_obs - Data_summary['NObs_Purged'].sum()} \n")
print("(4) Rainfall data observed period: \n")
print(Data_Composition.head())
print(Data_Composition.tail())
print(f"\n (5) Rainfall data mean observed period: {round(Data_Composition['ObservedPeriod(#Year)'].mean(),1)} year")

(1) NA value percentage (%): 0.0 % 

(2-1) Total number of observations (From Summary table sum-up): 269481) 

(2-2) Total number of observations (From Aggregated data number of row counting): 257639 

(3) The number of observations after filter application (From Summary table sum-up deducted the # of filtered lines): 257639 

(4) Rainfall data observed period: 

  Observatory   NObs  Percentage(%)To_TotalOBS  ObservedPeriod(#Year)
0         HKO  48947                     18.16                    144
1         SHA  14764                      5.48                     41
2         LFS  14399                      5.34                     40
3         TKL  14368                      5.33                     40
4         WGL  12968                      4.81                     37
   Observatory  NObs  Percentage(%)To_TotalOBS  ObservedPeriod(#Year)
19         TU1  6637                      2.46                     19
20         SKW  6364                      2.36                     18
21  

For Rainfall data, the longest observed period is 144 years for Hong Kong Observatory, same as for Temperature data. Mean period of observation is slightly longer, 32 years.

In [164]:
print("(6) Missing values by Observatory: \n")
print(Data_summary.iloc[:,[0,1,3,5,7,8]].sort_values(by='Tot_Purge_Pct(%)', ascending=False).head())
print("\n...\n")
print(Data_summary.iloc[:,[0,1,3,5,7,8]].sort_values(by='Tot_Purge_Pct(%)', ascending=False).tail())
print(f"\n (7) Percentage of observatories with less than 1% of missing values: {round(100*Data_summary.loc[Data_summary['Tot_Purge_Pct(%)']<1].shape[0]/Data_summary.shape[0],0)} %")

(6) Missing values by Observatory: 

  Dist  NObs_Dist  NObs_Starred  NObs_Trace  Starred_Pct(%)  Tot_Purge_Pct(%)
0  HKO      48947             1        6856            0.00             14.02
0  TMS      10258           586           0            5.71              5.74
0  WGL      12968           701           0            5.41              5.43
0  SEK      10320           493           0            4.78              4.81
0  PLC      11750           507           0            4.31              4.34

...

  Dist  NObs_Dist  NObs_Starred  NObs_Trace  Starred_Pct(%)  Tot_Purge_Pct(%)
0  PEN       7581            26           0            0.34              0.38
0  TWN       6882            17           0            0.25              0.29
0  CPH       8220            10           0            0.12              0.16
0   KP      11934            11           0            0.09              0.12
0  SSH       7520             4           0            0.05              0.09

 (7) Percentage of o

For Rainfall dataset, the overall missing value portion is similar to two other datasets, around 5% for the maximal case (Tai Mo Shan dataset (TMS)). The only difference is the lines containing 'Trace' value in Hong Kong Observatory dataset. 

Now we save the aggregated Rainfall dataset in Wrangled directory:

In [ ]:
treated_data_rep = r'../0_Data/wrangled/' 
if not os.path.exists(treated_data_rep):
    os.makedirs(treated_data_rep)
Agg_Data.to_pickle(treated_data_rep+"Rainfall_AGG.pkl")

# (4) Creation of an aggregated dataset with the daily average value of three variables 

In [26]:
treated_data_rep = r'../0_Data/wrangled/' 
Temperature_AGG = pd.read_pickle(treated_data_rep+"Temperature_AGG.pkl")
Rainfall_AGG = pd.read_pickle(treated_data_rep+"Rainfall_AGG.pkl")
Humidity_AGG = pd.read_pickle(treated_data_rep+"Humidity_AGG.pkl")
Temperature_AGG.set_index(["Date", "Dist"], inplace = True)
Rainfall_AGG.set_index(["Date", "Dist"], inplace = True)
Humidity_AGG.set_index(["Date", "Dist"], inplace = True)

In [27]:
AGG_temp =Temperature_AGG[['value']].join(Rainfall_AGG[['value']], 
                                          on = ["Date", "Dist"], 
                                          rsuffix = "RR").join(
                                              Humidity_AGG[['value']], 
                                              on = ["Date", "Dist"],
                                              rsuffix = "UM")

In [28]:
# Daily Average over observatories:
AGG_temp.reset_index(inplace = True)

In [29]:
del AGG_temp['Dist']
AGG_temp = AGG_temp.groupby('Date').mean()

In [31]:
AGG_temp.columns = ['TX', 'RR', 'UM']

In [33]:
treated_data_rep = r'../0_Data/wrangled/' 
if not os.path.exists(treated_data_rep):
    os.makedirs(treated_data_rep)
AGG_temp.to_pickle(treated_data_rep+"HKDaily_AGG.pkl")